<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EC%A6%9D%EB%B6%84/(%EB%85%B8%EC%9D%B4)_%EC%9B%90%EB%B3%B8H%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A6%9D%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본			   G_inc_so_model_노이2.pt
 산업분류자동화		   G_inc_so_model_노이.pt
'Colab Notebooks'	  'My Drive'
 G_inc_again_so_model.pt   Screenshot_20220401-001411_Chrome.jpg


In [24]:
import pandas as pd
df=pd.read_csv('/mydrive/산업분류자동화/dataset/spell_check/H_spell_check.csv', sep = ",", encoding = "UTF-8")

## 데이터 전처리

In [25]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000018,H,52,529,"선박, 화물 운송화물 검수"
1,id_0000043,H,49,493,개별화물자동차로 고객의 요청으로 화물 운송서비스
2,id_0000061,H,49,493,용달화물자동차로 고객의 요청으로 화물 운송서비스
3,id_0000063,H,52,529,시청에서 주민의 편의를 위해 주차장 운용
4,id_0000103,H,49,493,개별화물자동차로 고객의 요청으로 화물 운송서비스
...,...,...,...,...,...
98033,id_0999948,H,49,494,사업장에서 도시 내에서 소화물을 물류 배달 제공
98034,id_0999961,H,49,493,용달화물자동차로 고객의 요청으로 화물 운송서비스
98035,id_0999962,H,49,492,개인택시로 일반인을 대상으로 승객 운송서비스
98036,id_0999982,H,49,493,일반화물자동차로 고객의 요청으로 화물 운송서비스


In [26]:
df['digit_3'].value_counts().sort_index()

491       58
492    39176
493    49115
494     2134
495        3
501      281
502       34
511       32
512        9
521     1591
529     5605
Name: digit_3, dtype: int64

## **# 데이터 증분**

In [27]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [28]:
df0=df.loc[(df['digit_3'] == 491)]  #*50
df1=df.loc[(df['digit_3'] == 492)] 
df2=df.loc[(df['digit_3'] == 493)]  
df3=df.loc[(df['digit_3'] == 494)]  
df4=df.loc[(df['digit_3'] == 495)]   #*500
df5=df.loc[(df['digit_3'] == 501)]    #*10
df6=df.loc[(df['digit_3'] == 502)]    #*50
df7=df.loc[(df['digit_3'] == 511)]    #*50
df8=df.loc[(df['digit_3'] == 512)]   #*500
df9=df.loc[(df['digit_3'] == 521)]   
df10=df.loc[(df['digit_3'] == 529)]  

In [29]:
df1=df1.reset_index(drop=True)
df2=df2.reset_index(drop=True)
df3=df3.reset_index(drop=True)
df4=df4.reset_index(drop=True)
df5=df5.reset_index(drop=True)
df6=df6.reset_index(drop=True)
df7=df7.reset_index(drop=True)
df8=df8.reset_index(drop=True)
df9=df9.reset_index(drop=True)
df10=df10.reset_index(drop=True)

In [30]:
df_1=pd.concat([df1, df2, df3, df9, df10]) # *1
df_10=pd.concat([df5]) # *10
df_50=pd.concat([df0, df6, df7]) # *50 묶음
df_500=pd.concat([df4, df8]) # *500

In [31]:
df_1=df_1.reset_index(drop=True)
df_10=df_10.reset_index(drop=True)
df_50=df_50.reset_index(drop=True)
df_500=df_500.reset_index(drop=True)

In [32]:
df1.head()

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000119,H,49,492,개인택시로 일반인을 대상으로 승객 운송서비스
1,id_0000120,H,49,492,개인택시로 일반인을 대상으로 승객 운송서비스
2,id_0000147,H,49,492,개인택시로 일반인을 대상으로 승객 운송서비스
3,id_0000159,H,49,492,개인택시로 일반인을 대상으로 승객 운송서비스
4,id_0000173,H,49,492,개인택시로 일반인을 대상으로 승객 운송서비스


In [33]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df_10['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=9)
  id_list=[str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n])]
  digit_1_list=[df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n]]
  digit_2_list=[df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n]]
  digit_3_list=[df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n]]

  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_10_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df_10_to_insert, ignore_index=True)
  n=n+1

df_10=temp

In [34]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df_50['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=49)
  id_list=[str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n])]
  digit_1_list=[df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n]]
  digit_2_list=[df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n]]
  digit_3_list=[df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n]]

  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_50_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df_50_to_insert, ignore_index=True)
  n=n+1

df_50=temp

In [35]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df_500['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=499)
  id_list=[str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n]), 'p'+str(df_500['AI_id'][n])]
  digit_1_list=[df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n], df_500['digit_1'][n]]
  digit_2_list=[df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n], df_500['digit_2'][n]]
  digit_3_list=[df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n], df_500['digit_3'][n]]


  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_500_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df_500_to_insert, ignore_index=True)
  n=n+1

df_500=temp

기존 데이터 합치기

In [36]:
df=pd.concat([df_1, df_10, df_50, df_500])

In [37]:
df['digit_3'].value_counts().sort_index()

491.0     2900
492.0    39176
493.0    49115
494.0     2134
495.0     1500
501.0     2810
502.0     1700
511.0     1600
512.0     4500
521.0     1591
529.0     5605
Name: digit_3, dtype: int64

In [38]:
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000119,H,49.0,492.0,개인택시로 일반인을 대상으로 승객 운송서비스
1,id_0000120,H,49.0,492.0,개인택시로 일반인을 대상으로 승객 운송서비스
2,id_0000147,H,49.0,492.0,개인택시로 일반인을 대상으로 승객 운송서비스
3,id_0000159,H,49.0,492.0,개인택시로 일반인을 대상으로 승객 운송서비스
4,id_0000173,H,49.0,492.0,개인택시로 일반인을 대상으로 승객 운송서비스
...,...,...,...,...,...
5995,pid_0840437,H,51.0,512.0,운송서비스 해외 고객에게 사업장에서
5996,pid_0840437,H,51.0,512.0,사업장에서 고객에게 해외 운송서비스
5997,pid_0840437,H,51.0,512.0,사업장에서 해외
5998,pid_0840437,H,51.0,512.0,고객에게 해외


In [39]:
df.to_csv('df_H_inc.csv', index=False, encoding='utf-8-sig')